# M03A

M03A資料可作為統計通過門架ID對應的**通過量**  
可以分析路段的道路服務水準

## Setup

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import subprocess
import shutil
import tarfile
import xml.etree.ElementTree as ET
import re

/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [39]:
def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return os.path.abspath(folder_name)

def delete_folders(deletelist):
    """
    刪除資料夾
    deletelist(list):需要為皆為路徑的list
    """
    for folder_name in deletelist: 
        if os.path.exists(folder_name): # 檢查資料夾是否存在
            shutil.rmtree(folder_name) # 刪除資料夾及其內容
        else:
            print(f"資料夾 '{folder_name}' 不存在。")

def getdatelist(time1, time2):
    '''
    建立日期清單
    time1、time2(str):為%Y-%M-%D格式的日期字串
    '''
    if time1 > time2:
        starttime = time2
        endtime = time1
    else:
        starttime = time1
        endtime = time2

    date_range = pd.date_range(start=starttime, end=endtime)
    datelist = [d.strftime("%Y%m%d") for d in date_range]
    return datelist

def freewaydatafolder(datatype):
    savelocation = create_folder(os.path.join(os.getcwd(), datatype))
    rawdatafolder = create_folder(os.path.join(savelocation, '0_rawdata'))
    mergefolder = create_folder(os.path.join(savelocation, '1_merge'))
    excelfolder = create_folder(os.path.join(savelocation, '2_excel'))
    return rawdatafolder, mergefolder, excelfolder

def delete_folders_permanently(deletelist):
    """
    永久刪除資料夾及其內容，不放入資源回收筒
    deletelist (list): 需要刪除的資料夾路徑列表
    """
    for item in deletelist:
        if os.path.isdir(item):  # 檢查是否為資料夾
            try:
                shutil.rmtree(item)  # 永久刪除資料夾
                print(f"已永久刪除資料夾： {item}")
            except OSError as e:
                print(f"刪除資料夾 {item} 時發生錯誤： {e}")
        elif os.path.isfile(item):  # 檢查是否為檔案
            try:
                os.remove(item)  # 永久刪除檔案
                print(f"已永久刪除檔案： {item}")
            except OSError as e:
                print(f"刪除檔案 {item} 時發生錯誤： {e}")
        else:
            print(f"{item} 不是檔案或資料夾。")

def download_etag(etagurl, etagdownloadpath):
    """
    下載指定網址的 XML 檔案到指定位置。

    Args:
        etagurl (str): 要下載的 XML 檔案網址。
        etagdownloadpath (str): 檔案下載後的儲存路徑（包含檔案名稱）。
    """

    try:
        response = requests.get(etagurl, stream=True)
        response.raise_for_status()  # 檢查 HTTP 狀態碼，如有錯誤則拋出異常

        with open(etagdownloadpath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

    except requests.exceptions.RequestException as e:
        print(f"下載時發生錯誤：{e}")
    except Exception as e:
        print(f"發生錯誤：{e}")

def read_xml(xml_file_path):
    """
    讀取並解析 XML 檔案。

    Args:
        xml_file_path (str): XML 檔案路徑。

    Returns:
        ElementTree.Element: XML 文件的根節點。
        None: 如果解析失敗。
    """
    # try:
    #     tree = ET.parse(xml_file_path)
    #     root = tree.getroot()
    #     return root
    try:
        with open(xml_file_path, 'r', encoding='utf-8') as f:  # 指定編碼
            xml_content = f.read()
        return xml_content
    except FileNotFoundError:
        print(f"檔案未找到：{xml_file_path}")
        return None
    except ET.ParseError as e:
        print(f"解析 XML 檔案時發生錯誤：{e}")
        return None

def etag_xml_to_dataframe(xml_content):
    """
    將 XML 內容轉換為 Pandas DataFrame。

    Args:
        xml_content (str): XML 內容字串。

    Returns:
        pandas.DataFrame: 轉換後的 DataFrame。
        None: 如果解析失敗。
    """
    try:
        root = ET.fromstring(xml_content)  # 從字串解析 XML

        data = []
        for etag in root.findall('.//{http://traffic.transportdata.tw/standard/traffic/schema/}ETag'):
            etag_data = {}
            for element in etag:
                tag_name = element.tag.split('}')[-1]  # 去除命名空間
                if tag_name == 'RoadSection':  # 處理 RoadSection
                    for section_element in element:
                        etag_data[section_element.tag] = section_element.text
                else:
                    etag_data[tag_name] = element.text
            data.append(etag_data)

        df = pd.DataFrame(data)
        df.columns = ['ETagGantryID','LinkID', 'LocationType', 'PositionLon', 'PositionLat', 'RoadID', 'RoadName', 'RoadClass', 'RoadDirection', 'Start','End', 'LocationMile']
        return df

    except ET.ParseError as e:
        print(f"解析 XML 內容時發生錯誤：{e}")
        return None
    except Exception as e:
        print(f"發生錯誤：{e}")
        return None

def etag_getdf():
    etagfolder = create_folder(os.path.join(os.getcwd(), 'ETag'))
    etagurl = 'https://tisvcloud.freeway.gov.tw/history/motc20/ETag.xml'
    etagdownloadpath = os.path.join(etagfolder, 'ETag.xml')
    download_etag(etagurl=etagurl, etagdownloadpath=etagdownloadpath)
    etagxml = read_xml(etagdownloadpath)
    etag = etag_xml_to_dataframe(etagxml)

    etag.to_excel(os.path.join(etagfolder,'Etag.xlsx'), index = False)
    return etag

def extract_tar_gz(tar_gz_file, extract_path):
    try:
        with tarfile.open(tar_gz_file, 'r:gz') as tar:
            tar.extractall(path=extract_path)
    except Exception as e:
        print(f"解壓縮 {tar_gz_file} 失敗：{e}")

def download_and_extract(url, datatype, date, downloadfolder, keep = False):
    '''針對高公局交通資料庫的格式進行下載'''
    downloadurl = f"{url}/{datatype}_{date}.tar.gz"
    destfile = os.path.join(downloadfolder, f"{datatype}_{date}.tar.gz")

    response = requests.get(downloadurl)
    with open(destfile, 'wb') as file:
        file.write(response.content)

    extractpath = create_folder(os.path.join(downloadfolder, date))
    extract_tar_gz(destfile, extractpath)
    if keep == False:
        os.remove(destfile)

    return extractpath

def findfiles(filefolderpath, filetype='.csv'):
    """
    尋找指定路徑下指定類型的檔案，並返回檔案路徑列表。

    Args:
        filefolderpath (str): 指定的檔案路徑。
        filetype (str, optional): 要尋找的檔案類型，預設為 '.csv'。

    Returns:
        list: 包含所有符合條件的檔案路徑的列表。
    """

    filelist = []  # 建立一個空列表來儲存檔案路徑

    # 使用 os.walk 遍歷資料夾及其子資料夾
    for root, _, files in os.walk(filefolderpath):
        for file in files:
            if file.endswith(filetype):  # 檢查檔案是否以指定類型結尾
                file_path = os.path.join(root, file)  # 建立完整的檔案路徑
                filelist.append(file_path)  # 將檔案路徑添加到列表中

    return filelist

def combinefile(filelist, datatype='M03A'):
    """
    更有效率地合併多個CSV檔案。

    Args:
        filelist (list): 包含CSV檔案路徑的列表。
        datatype (str, optional): 資料類型，決定欄位名稱。預設為 'M03A'。

    Returns:
        pandas.DataFrame: 合併後的DataFrame。
    """

    # 使用字典來映射資料類型和欄位名稱，避免重複的 if/elif 判斷
    column_mapping = {
        'M03A': ['TimeStamp', 'GantryID', 'Direction', 'VehicleType', 'Volume'],
        'M04A': ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'TravelTime', 'Volume'],
        'M05A': ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'Speed', 'Volume'],
        'M06A': ['VehicleType', 'DetectionTimeO', 'GantryO', 'DetectionTimeD', 'GantryD', 'TripLength', 'TripEnd', 'TripInformation'],
        'M07A': ['TimeStamp', 'GantryO', 'VehicleType', 'AverageTripLength', 'Volume'],
        'M08A': ['TimeStamp', 'GantryO', 'GantryD', 'VehicleType', 'Trips']
    }

    columns = column_mapping.get(datatype)  # 使用 get() 方法，如果找不到鍵，會返回 None
    if columns is None:
        raise ValueError(f"未知的資料類型：{datatype}")

    combineddf = pd.concat(
        (pd.read_csv(i, header=None, names=columns) for i in filelist),  # 使用生成器表達式
        ignore_index=True  # 避免重複的索引
    )

    return combineddf

def THI_M03A(df):
    df = df.pivot(index=['TimeStamp', 'GantryID', 'Direction'], columns='VehicleType', values='Volume').reset_index()
    df = df.rename(columns = {
        5 : 'Vol_Trail',
        31 : 'Vol_Car',
        32 : 'Vol_Truck',
        41 : 'Vol_TourBus',
        42 : 'Vol_BTruck'
    })
    df = df.reindex(columns = ['TimeStamp', 'GantryID', 'Direction', 'Vol_Trail', 'Vol_Car', 'Vol_Truck', 'Vol_TourBus', 'Vol_BTruck'])

    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])

    df['Date'] = df['TimeStamp'].dt.date
    df['Hour'] = df['TimeStamp'].dt.hour

    df = df.groupby(['Date','Hour','GantryID','Direction']).agg({
            'Vol_Trail':'sum',
            'Vol_Car':'sum', 
            'Vol_Truck':'sum',
            'Vol_TourBus':'sum',
            'Vol_BTruck':'sum'}).reset_index()
    return df

def THI_M05A(df, weighted = False):
    
    # 將每5分鐘的資料，轉為分時資料
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['Date'] = df['TimeStamp'].dt.date
    df['Hour'] = df['TimeStamp'].dt.hour

    df = df[df['Volume']!=0] # 需要避開Volume 為0的資料

    if weighted == True:
        df['Speed_time_volume'] = df['Speed'] * df['Volume']
        df = df.groupby(['Date', 'Hour', 'GantryFrom', 'GantryTo', 'VehicleType']).agg({'Speed_time_volume':'sum', 'Volume':'sum'}).reset_index()
        df['Speed'] = df['Speed_time_volume'] / df['Volume']
    else :
        df = df.groupby(['Date', 'Hour', 'GantryFrom', 'GantryTo', 'VehicleType']).agg({'Speed':'mean'}).reset_index()
    
    
    df['Speed'] = df['Speed'].round(3)
    df = df.pivot(index=['Date', 'Hour', 'GantryFrom', 'GantryTo'], columns='VehicleType', values='Speed').reset_index()
    df = df.rename(columns = {
        5 : 'Speed_Trail',
        31 : 'Speed_Car',
        32 : 'Speed_Truck',
        41 : 'Speed_TourBus',
        42 : 'Speed_BTruck'
    })

    df = df.fillna(0)
    df = df.reindex(columns = ['Date', 'Hour', 'GantryFrom', 'GantryTo', 'Speed_Trail', 'Speed_Car', 'Speed_Truck', 'Speed_TourBus', 'Speed_BTruck'])
    
    return df

def THI_M06A(df, hour = True):
    df['DetectionTimeO'] = pd.to_datetime(df['DetectionTimeO'])
    df['DetectionTimeD'] = pd.to_datetime(df['DetectionTimeD'])

    df['Date'] = df['DetectionTimeO'].dt.date
    if hour == True:
        df['HourO'] = df['DetectionTimeO'].dt.hour
        df['HourD'] = df['DetectionTimeD'].dt.hour
        df = df.groupby(['Date','HourO', 'GantryO', 'HourD', 'GantryD', 'VehicleType']).size().reset_index(name='Volume')

    df = df.groupby(['Date', 'GantryO', 'GantryD','VehicleType']).size().reset_index(name='Volume')

    return df 

In [3]:
# 還沒有完成的
# def THI_M06A(df):

In [40]:
def THI_process(df, datatype, weighted = False, hour = True):
    if datatype == 'M03A':
        df = THI_M03A(df)
    elif datatype == 'M05A':
        df = THI_M05A(df, weighted = weighted)
    elif datatype == 'M06A':
        df = THI_M06A(df, hour = hour)
    return df

In [5]:
def M03A_Tableau_combined(folder , etag):
    allfiles = findfiles(filefolderpath=folder, filetype='.xlsx')
    combineddf = pd.concat(
        (pd.read_excel(i) for i in allfiles),  # 使用生成器表達式
        ignore_index=True  # 避免重複的索引
    )

    combineddf['Day'] = combineddf["Date"].dt.day_name() #生成星期幾

    combineddf = pd.merge(combineddf,etag[['ETagGantryID', 'RoadName','Start', 'End']].rename(columns = {'ETagGantryID':'GantryID'}) , on = 'GantryID')
    combineddf['RoadSection'] = combineddf['Start'] + '-' + combineddf['End']

    outputfolder = create_folder(os.path.join(folder, '..', '3_TableauData'))
    combineddf.to_csv(os.path.join(outputfolder, 'M03A.csv'), index=False)

In [6]:
'''
待改進
def combinefile(filelist, datatype = 'M03A'):
    if datatype == 'M03A':
        columns = ['TimeStamp', 'GantryID', 'Direction', 'VehicleType', 'Volume']
    elif datatype == 'M04A':
        columns = ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'TravelTime', 'Volume']
    elif datatype == 'M05A':
        columns = ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'Speed', 'Volume']
    elif datatype == 'M06A':
        columns = ['VehicleType', 'DetectionTimeO', 'GantryO',  'DetectionTimeD', 'GantryD', 'TripLength', 'TripEnd', 'TripInformation']
    elif datatype == 'M07A':
        columns = ['TimeStamp', 'GantryO', 'VehicleType', 'AverageTripLength', 'Volume']
    elif datatype == 'M08A':
        columns = ['TimeStamp',	'GantryO', 'GantryD', 'VehicleType', 'Trips']
    
    combineddf = []
    for i in filelist:
        df = pd.read_csv(i, header=None)
        df.columns = columns
        combineddf.append(df)
    combineddf = pd.concat(combineddf)

    return combineddf
'''

"\n待改進\ndef combinefile(filelist, datatype = 'M03A'):\n    if datatype == 'M03A':\n        columns = ['TimeStamp', 'GantryID', 'Direction', 'VehicleType', 'Volume']\n    elif datatype == 'M04A':\n        columns = ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'TravelTime', 'Volume']\n    elif datatype == 'M05A':\n        columns = ['TimeStamp', 'GantryFrom', 'GantryTo', 'VehicleType', 'Speed', 'Volume']\n    elif datatype == 'M06A':\n        columns = ['VehicleType', 'DetectionTimeO', 'GantryO',  'DetectionTimeD', 'GantryD', 'TripLength', 'TripEnd', 'TripInformation']\n    elif datatype == 'M07A':\n        columns = ['TimeStamp', 'GantryO', 'VehicleType', 'AverageTripLength', 'Volume']\n    elif datatype == 'M08A':\n        columns = ['TimeStamp',\t'GantryO', 'GantryD', 'VehicleType', 'Trips']\n    \n    combineddf = []\n    for i in filelist:\n        df = pd.read_csv(i, header=None)\n        df.columns = columns\n        combineddf.append(df)\n    combineddf = pd.concat(comb

## 需要調整的參數

In [7]:
# ===== Step 0: 手動需要調整的參數 =====

# 需要調整的項目有2個
# 1. 調整需要確認下載的資料型態是什麼
# datatype = "M03A"  # Data type (e.g., M03A, M06A, M05A) 

# 2. 調整下載的資料區間
starttime = "2024-09-10"
endtime = "2024-09-10"
datelist = getdatelist(endtime,starttime) # 下載的時間區間清單

PCE = {'PCE5':3, 
        'PCE31':1,
        'PCE32':1,
        'PCE41':1.8
        'PCE42':1.8}

# 建立後續要處理儲存資料的資料夾位置
# savelocation, rawdatafolder, mergefolder, excelfolder = freewaydatafolder(datatype=datatype)
# savelocation = create_folder(os.path.join(os.getcwd(), datatype))
# rawdatafolder = create_folder(os.path.join(savelocation, '0_rawdata'))
# mergefolder = create_folder(os.path.join(savelocation, '1_merge'))
# excelfolder = create_folder(os.path.join(savelocation, '2_excel'))
# basicurl = "https://tisvcloud.freeway.gov.tw/history/TDCS/"
# url = basicurl + datatype

## 程式執行

In [20]:
def freeway(datatype, datelist, Tableau = False, etag = None, hour = True):
    rawdatafolder, mergefolder, excelfolder = freewaydatafolder(datatype=datatype)
    url = "https://tisvcloud.freeway.gov.tw/history/TDCS/" + datatype

    for date in datelist :
        # 1. 下載並解壓縮
        dowloadfilefolder = download_and_extract(url = url, datatype = datatype, date = date, downloadfolder = rawdatafolder)

        # 2. 合併
        filelist = findfiles(filefolderpath=dowloadfilefolder, filetype='.csv')
        df = combinefile(filelist=filelist, datatype=datatype)
        mergeoutputfolder = create_folder(os.path.join(mergefolder, date)) # 建立相同日期的資料夾進行處理
        df.to_csv(os.path.join(mergeoutputfolder, f'{date}.csv') , index = False) # 輸出整併過的csv
        delete_folders([dowloadfilefolder]) #回頭刪除解壓縮過的資料

        # # 3. 處理
        # df = THI_process(df, datatype=datatype)
        # df.to_excel(os.path.join(excelfolder, f'{date}.xlsx'), index = False, sheet_name = date)
    
    if Tableau == True:
        if datatype == 'M03A':
            M03A_Tableau_combined(folder=excelfolder, etag = etag)

    return df

In [33]:
datatype = 'M06A'
rawdatafolder, mergefolder, excelfolder = freewaydatafolder(datatype=datatype)

dfs = []
for date in datelist:
    path = os.path.join(mergefolder,date,f'{date}.csv')
    df = pd.read_csv(path)
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
def M06A_Ramp(datelist):
    df = pd.concat(pd.read_csv(os.path.join(mergefolder, date, f'{date}.csv')) for date in datelist)


In [36]:
df = pd.concat(pd.read_csv(os.path.join(mergefolder, date, f'{date}.csv')) for date in datelist)


In [38]:
df

,VehicleType,DetectionTimeO,GantryO,DetectionTimeD,GantryD,TripLength,TripEnd,TripInformation
0,32,2024-09-10 03:27:26,03F2260N,2024-09-10 03:37:57,03F2125N,17.30,Y,2024-09-10 03:27:26+03F2260N; 2024-09-10 03:29...
1,5,2024-09-10 03:01:17,01F3535N,2024-09-10 03:37:35,01F3019N,56.63,Y,2024-09-10 03:01:17+01F3535N; 2024-09-10 03:06...
2,31,2024-09-10 03:19:16,03F0158S,2024-09-10 03:38:48,05F0309S,39.80,Y,2024-09-10 03:19:16+03F0158S; 2024-09-10 03:20...
3,5,2024-09-10 03:38:05,01F3696N,2024-09-10 03:50:00,01F3535N,21.00,Y,2024-09-10 03:38:05+01F3696N; 2024-09-10 03:39...
4,32,2024-09-10 03:29:28,01F0376N,2024-09-10 03:32:02,01F0340N,8.40,Y,2024-09-10 03:29:28+01F0376N; 2024-09-10 03:32...
...,...,...,...,...,...,...,...,...
3324892,31,2024-09-10 22:49:29,01F3286S,2024-09-10 23:13:24,01F3640S,40.00,Y,2024-09-10 22:49:29+01F3286S; 2024-09-10 22:54...
3324893,31,2024-09-10 22:46:25,03F0116N,2024-09-10 22:53:07,03F0021N,12.70,Y,2024-09-10 22:46:25+03F0116N; 2024-09-10 22:50...
3324894,31,2024-09-10 22:20:20,01F0005N,2024-09-10 22:20:20,01F0005N,1.10,Y,2024-09-10 22:20:20+01F0005N
3324895,31,2024-09-10 22:31:51,01F3640N,2024-09-10 22:31:51,01F3640N,5.00,Y,2024-09-10 22:31:51+01F3640N


In [28]:
df = freeway(datatype = 'M06A', datelist = datelist)

In [30]:
# 竹林交流道對應表
ramp_data = {
    "Name": ["南出匝道", "南入匝道", "北出匝道", "北入匝道"],
    "Pass": ["03F0846S", "03F0961S", "03F0961N", "03F0846N"],
    "UnPass": ["03F0961S", "03F0846S", "03F0846N", "03F0961N"]
}
Ramp = pd.DataFrame(ramp_data)




for index, row in Ramp.iterrows():
    pass_gantry = row["Pass"]
    unpass_gantry = row["UnPass"]
    ramp_name = row["Name"]
    
    # 過濾有經過 A 但沒有經過 B 的車輛
    df_part = df[df["TripInformation"].str.contains(pass_gantry, na=False) & ~df["TripInformation"].str.contains(unpass_gantry, na=False)]
    
    # 解析 PassHour
    pass_hours = []
    for trip_info in df_part["TripInformation"]:
        times = trip_info.split('; ')
        matched_times = [t for t in times if pass_gantry in t]
        
        if matched_times:
            time_match = re.search(r"\d{2}:\d{2}:\d{2}", matched_times[0])
            if time_match:
                pass_hour = datetime.strptime(time_match.group(), "%H:%M:%S").hour
                pass_hours.append(pass_hour)
            else:
                pass_hours.append(None)
        else:
            pass_hours.append(None)
    
    df_part["PassHour"] = pass_hours
    
    # 依 VehicleType 和 PassHour 分組統計
    df_part_g = df_part.groupby(["VehicleType", "PassHour"], as_index=False).size()
    df_part_g["Ramp"] = ramp_name
    
    ramp_df = pd.concat([ramp_df, df_part_g], ignore_index=True)

/var/folders/1p/d63vtkgd60d4sbpl791031gw0000gn/T/ipykernel_11260/3992791163.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part["PassHour"] = pass_hours


NameError: name 'ramp_df' is not defined

In [ ]:
# ramp_data = {
#     "Name": ["南出匝道", "南入匝道", "北出匝道", "北入匝道"],
#     "Pass": ["03F0846S", "03F0961S", "03F0961N", "03F0846N"],
#     "UnPass": ["03F0961S", "03F0846S", "03F0846N", "03F0961N"]
# }
# Ramp = pd.DataFrame(ramp_data)

# for _, row in Ramp.iterrows():
#     pass_gantry = row['Pass']
#     unpass_gantry = row['UnPass']
#     ramp_name = row['Name']
    
#     # 篩選有經過 Pass 但未經過 UnPass 的車輛
#     df_part = df[df['TripInformation'].str.contains(pass_gantry) & ~df['TripInformation'].str.contains(unpass_gantry)]
    
#     # 判斷通過小時
#     pass_hours = []
#     for trip_info in df_part['TripInformation']:
#         times = trip_info.split('; ')
#         matched_times = [t for t in times if pass_gantry in t]
#         if matched_times:
#             pass_hour = datetime.strptime(matched_times[0], "%H:%M:%S").hour
#             pass_hours.append(pass_hour)
#         else:
#             pass_hours.append(None)
    
#     df_part['PassHour'] = pass_hours
    
#     # 進行 groupby 統計
#     df_part_g = df_part.groupby(["VehicleType", "PassHour"], as_index=False).size()
#     df_part_g.rename(columns={'size': 'Volume'}, inplace=True)
#     df_part_g['Ramp'] = ramp_name
#     df_part_g['date'] = file_name.replace(".csv", "")
    
#     ramp_df = pd.concat([ramp_df, df_part_g], ignore_index=True)


ValueError: time data '2024-09-10 03:47:24+03F0846S' does not match format '%H:%M:%S'

In [ ]:
passgantry = {'Gantry_upstream':'01F1389N', # 銅鑼 -> 苗栗
              'Gantry_downstream':'01F1465N', 
              'Interchange':'' }

In [ ]:
import os
import pandas as pd
import glob
from datetime import datetime
from openpyxl import Workbook

# 設定輸出路徑
savelocation_output = os.path.join(location, data_type, "2_output")
os.makedirs(savelocation_output, exist_ok=True)

# 定義 Ramp 資料
ramp_data = {
    "Name": ["南出匝道", "南入匝道", "北出匝道", "北入匝道"],
    "Pass": ["03F0846S", "03F0961S", "03F0961N", "03F0846N"],
    "UnPass": ["03F0961S", "03F0846S", "03F0846N", "03F0961N"]
}
Ramp = pd.DataFrame(ramp_data)

# 讀取合併後的檔案
file_list = glob.glob(os.path.join(savelocation_merge, "*.csv"))
ramp_df = pd.DataFrame(columns=["VehicleType", "PassHour", "Volume", "Ramp", "date"])

if file_list:
    file_path = file_list[0]  # 只處理第一個文件
    file_name = os.path.basename(file_path)
    print(f"=== {file_name} ===")
    
    df = pd.read_csv(file_path, dtype=str)  # 讀取 CSV
    
    for _, row in Ramp.iterrows():
        pass_gantry = row['Pass']
        unpass_gantry = row['UnPass']
        ramp_name = row['Name']
        
        # 篩選有經過 Pass 但未經過 UnPass 的車輛
        df_part = df[df['TripInformation'].str.contains(pass_gantry) & ~df['TripInformation'].str.contains(unpass_gantry)]
        
        # 判斷通過小時
        pass_hours = []
        for trip_info in df_part['TripInformation']:
            times = trip_info.split('; ')
            matched_times = [t for t in times if pass_gantry in t]
            if matched_times:
                pass_hour = datetime.strptime(matched_times[0], "%H:%M:%S").hour
                pass_hours.append(pass_hour)
            else:
                pass_hours.append(None)
        
        df_part['PassHour'] = pass_hours
        
        # 進行 groupby 統計
        df_part_g = df_part.groupby(["VehicleType", "PassHour"], as_index=False).size()
        df_part_g.rename(columns={'size': 'Volume'}, inplace=True)
        df_part_g['Ramp'] = ramp_name
        df_part_g['date'] = file_name.replace(".csv", "")
        
        ramp_df = pd.concat([ramp_df, df_part_g], ignore_index=True)
    
    # 儲存為 Excel
    output_path = os.path.join(savelocation_output, f"竹林交流道交通量_{file_name.replace('.csv', '')}.xlsx")
    ramp_df.to_excel(output_path, index=False, engine='openpyxl')
    print(f"檔案已儲存至: {output_path}")


In [24]:
etag_getdf()

,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile
0,03F2899N,0000300129000Q,4,120.48633,23.511683,000030,國道3號,0,N,竹崎(縣道159線),竹崎(縣道166線),289K+900
1,05F0000S,0000501001000A,4,121.62302,25.035183,000050,國道5號,0,S,南港系統,石碇,0K+000
2,03F2306N,0000300123000M,4,120.70283,23.90678,000030,國道3號,0,N,南投服務區,南投,230K+600
3,01F3227N,0000100132300D,4,120.25,23.014172,000010,國道1號,0,N,大灣,永康,322K+700
4,03F1991S,0000300019900N,4,120.57766,24.102808,000030,國道3號,0,S,彰化系統,快官,199K+100
...,...,...,...,...,...,...,...,...,...,...,...,...
334,01F2827S,0000100028300D,4,120.32219,23.34747,000010,國道1號,0,S,嘉義系統,新營服務區,282K+700
335,01F2394N,0000100123900P,4,120.474915,23.702602,000010,國道1號,0,N,斗南,虎尾,239K+400
336,03F1332N,0000300113300K,4,120.76442,24.579641,000030,國道3號,0,N,西湖服務區,後龍,133K+200
337,05F0439S,0000500004400G,4,121.78946,24.710684,000050,國道5號,0,S,宜蘭(壯圍),羅東,43K+900


In [18]:
def main():
    '''主要會用freeway這個函數進行三個步驟 (1) 下載 (2) 整併當日資料 (3) 處理
    請根據需要調整datatype(str)

    1. M03A : 主要計算主要路段通過門架的通過量
    2. M05A : 計算通過兩個門架間的速率
    3. M06A : 
             (1) 計算通過兩個門架之間的OD數量
             (2) 計算匝道出入量：通過兩個路段，但僅出現在第一段，卻沒有出現在第二段的量 
    '''
    

    etag = etag_getdf()
    
    # freeway(datatype = 'M03A', datelist = datelist) 
    # freeway(datatype = 'M05A', datelist = datelist)
    
    
    freeway(datatype = 'M06A', datelist = datelist, hour = True) # 計算OD:如果只是要全日的OD，就可以把"hour = True" 改為 "hour = False"
    freeway(datatype = 'M06A')

In [19]:
if __name__ == '__main__':
    main()

# 以備不時之需

In [133]:
# def main():
#     etag = etag_getdf()

#     for date in datelist :
#         # 1. 下載並解壓縮
#         dowloadfilefolder = download_and_extract(url = url, datatype = datatype, date = date, downloadfolder = rawdatafolder)

#         # 2. 合併
#         filelist = findfiles(filefolderpath=dowloadfilefolder, filetype='.csv')
#         df = combinefile(filelist=filelist, datatype=datatype)
#         mergeoutputfolder = create_folder(os.path.join(mergefolder, date)) # 建立相同日期的資料夾進行處理
#         df.to_csv(os.path.join(mergeoutputfolder, f'{date}.csv') , index = False) # 輸出整併過的csv
#         delete_folders([dowloadfilefolder]) #回頭刪除解壓縮過的資料

#         # 3. 處理
#         df = THI_process(df, datatype=datatype)
#         df.to_excel(os.path.join(excelfolder, f'{date}.xlsx'), index = False, sheet_name = date)

#     # M03A_Tableau_combined(folder=excelfolder, etag = etag)
